# Tutorial of NvDocker interfaces

In [1]:
from dsd.sys.docker import NvDocker
nvd = NvDocker()

# Gpu global informations

In [2]:
globalInfo = nvd.GpuGlobalInfo()

print('Driver_Version:', globalInfo['Driver_Version'])
print('Cuda_Version:', globalInfo['Cuda_Version'])

('Driver_Version:', u'346.46')
('Cuda_Version:', u'7.0')


# Gpu status

In [3]:
gpuInfo = nvd.GpuInfo()

for i in range(len(gpuInfo)):
    print('#'*20)
    print('Model:', gpuInfo[i]['Model'])
    print('Path:', gpuInfo[i]['Path'])
    print('Memory_Used:', gpuInfo[i]['Memory_Used'], 'MB')
    print('Memory_Total:', gpuInfo[i]['Memory_Total'], 'MB')
    
    print('Utilization_Gpu:', gpuInfo[i]['Utilization_Gpu'])
    print('Utilization_Memory:', gpuInfo[i]['Utilization_Memory'])
    print('Temperature:', gpuInfo[i]['Temperature'])
    
    for p in gpuInfo[i]['Processes']:
        print(' '*10, 'PID:', p['PID'], 'Name:', p['Name'], 'MemoryUsed:', p['MemoryUsed'], 'MB')

####################
('Model:', u'Graphics Device')
('Path:', u'/dev/nvidia0')
('Memory_Used:', 9860, 'MB')
('Memory_Total:', 12287, 'MB')
('Utilization_Gpu:', 76)
('Utilization_Memory:', 41)
('Temperature:', 82)
('          ', 'PID:', 5749, 'Name:', u'python', 'MemoryUsed:', 2718, 'MB')
('          ', 'PID:', 2673, 'Name:', u'python', 'MemoryUsed:', 127, 'MB')
('          ', 'PID:', 2988, 'Name:', u'python', 'MemoryUsed:', 127, 'MB')
('          ', 'PID:', 21497, 'Name:', u'python', 'MemoryUsed:', 127, 'MB')
('          ', 'PID:', 13504, 'Name:', u'/home/zhouyao/conda/anaconda/bin/python', 'MemoryUsed:', 1974, 'MB')
('          ', 'PID:', 24486, 'Name:', u'/home/zhouyao/conda/anaconda/bin/python', 'MemoryUsed:', 129, 'MB')
('          ', 'PID:', 10879, 'Name:', u'/home/zhouyao/conda/anaconda/bin/python', 'MemoryUsed:', 1972, 'MB')
('          ', 'PID:', 26769, 'Name:', u'/home/zhouyao/conda/anaconda/bin/python', 'MemoryUsed:', 306, 'MB')
('          ', 'PID:', 31240, 'Name:', u'python